### Project Setup


In [171]:
# %pip install --upgrade langchain langsmith langgraph langchain_openai

In [172]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get OPENAI_API_KEY from .env file
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_ORGANIZATION"] = os.getenv("OPENAI_ORGANIZATION")

# Initialize LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Agent Demo"

### Define the Agents


In [173]:
squad_members = ["ContextRetriever", "Coder", "Tester", "Saver"]
# squad_members = ["ContextRetriever"]

### Set up Agent 1: Context Retriever

In [174]:
# Create the tool for querying the vector database for relevant code snippets
from weaviate import Client
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool


class ContextRetrieverInput(BaseModel):
    query: str = Field(
        description="Exact, original description of the feature the user would like to build"
    )


# Initialize the Weaviate client
client = Client("http://localhost:8080")


def query_collection(query):
    response = (
        client.query.get("code_example", ["code"])
        .with_near_text({"concepts": [query]})
        .with_limit(9)
        .do()
    )
    return response


context_retriever = StructuredTool.from_function(
    func=query_collection,
    name="ContextRetriever",
    description="Retrieve documents relevant to the user's feature request",
    args_schema=ContextRetrieverInput,
    return_direct=False,
)

In [175]:
from langgraph.prebuilt import ToolExecutor

retriever_tools=[context_retriever]

retriever_tool_executor = ToolExecutor(retriever_tools)

In [176]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_openai import ChatOpenAI


def create_retriever_agent(tools: list):
    system_prompt_initial = """
    You are a research assistant who can search for relevant documents from a vector database.
    Work autonomously according to your specialty, using the tools available to you.
    Do not ask for clarification.
    You are chosen for a reason!
    You gather accurate information from a vector database based upon the user's query in its entirety. You will need to use your tool to fetch this information. You should pass in the entire user request. Once you have fetched the information, you can let the Human know that you have retrieved the context and the task is complete. Do not return the information to the Human, as they will not be able to understand it.
    """

    agent_llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessagePromptTemplate.from_template(system_prompt_initial),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(agent_llm, tools, prompt)
    # executor = AgentExecutor(agent=agent, tools=tools)
    return agent


context_retriever_agent = create_retriever_agent(retriever_tools)

In [177]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_openai import ChatOpenAI


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessagePromptTemplate.from_template(system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    # executor = AgentExecutor(agent=agent, tools=tools)
    return agent

In [178]:
from langchain_core.messages import HumanMessage


def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

### Create the Supervisor's Tools


In [179]:
from langchain.pydantic_v1 import BaseModel
from enum import Enum
from langchain.tools import StructuredTool

supervisor_options = ["FINISH"] + squad_members

RouteOptions = Enum("RouteOptions", {option: option for option in supervisor_options})


class RouteInput(BaseModel):
    next: RouteOptions


def route(route: str) -> str:
    return route


router = StructuredTool.from_function(
    func=route,
    name="route",
    description="Select the next role",
    args_schema=RouteInput,
    return_direct=False,
)

In [180]:
from langgraph.prebuilt import ToolExecutor

supervisor_tools=[router]
supervisor_tool_executor = ToolExecutor(supervisor_tools)

### Create the Agent's Tools

### Create the Supervisor

In [181]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function

# Define the system prompts
system_prompt_initial = """
You are a supervisor tasked with managing a conversation between the
following workers on a development team: {members}. Given the following feature request from a user, respond with the worker to act next.

Each worker will perform a task and respond with their results and status. This task is complete as soon as you know a worker has retrieved context related to the user's feature request. When the task is complete, respond with FINISH.

You should never call the Saver directly.
"""

system_prompt_final = """
Given the conversation above, who should act next?
Or should we FINISH? Remember, we FINISH as soon as you know that one of the workers has found the context for the user's query. Select one of: {options}
"""

# Using openai function calling can make output parsing easier for us
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt_initial),
        MessagesPlaceholder(variable_name="messages"),
        SystemMessagePromptTemplate.from_template(system_prompt_final),
    ]
).partial(options=str(supervisor_options), members=", ".join(squad_members))

# Initialize the LLM node
supervisor_llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

# Bind the tools to the LLM node
functions = [format_tool_to_openai_function(t) for t in supervisor_tools]

supervisor_chain = prompt | supervisor_llm.bind_functions(functions) | JsonOutputFunctionsParser()

In [182]:
# Test it out
from langchain_core.messages import HumanMessage

supervisor_chain.invoke(
    {
        "messages": [
            HumanMessage(content="I want to build a chain that calls an LLM and then parses the response with Pydantic"),
        ]
    }
)

{'next': 'ContextRetriever'}

### Define the nodes

In [183]:
from langchain_core.messages import HumanMessage

from langchain_core.messages import (
    FunctionMessage,
)


def call_retriever(state):
    messages = state["messages"]
    last_message = messages[-1]
    response = context_retriever_agent.invoke(state)
    print("Retriever response:")
    print(response)
    print(response['output'])
    print(isinstance(response, FunctionMessage))
    # We return a list, because this will get added to the existing list
    return {
        "messages": [HumanMessage(content=response["output"], name="Content Retriever")]
    }

### Construct the Graph

In [184]:
import operator
from typing import Annotated, Sequence, TypedDict
import functools

from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain_core.agents import AgentAction


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str
    # The 'context' field captures the code context so that it can always be remembered in its 100% accurate state
    context: str
    # The interediate steps field captures the steps taken by the agennt
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

agent_llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
# code_agent = create_agent(llm, [python_repl_tool], "You may generate safe python code to analyze data and generate charts using matplotlib.")
# code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

workflow = StateGraph(AgentState)
workflow.add_node("ContextRetriever", call_retriever)
workflow.add_node("Coder", call_retriever)
workflow.add_node("Tester", call_retriever)
workflow.add_node("Saver", call_retriever)
workflow.add_node("supervisor", supervisor_chain)

In [185]:
# We want our workers to ALWAYS "report back" to the supervisor when done
workflow.add_edge("ContextRetriever", "supervisor")
workflow.add_edge("Coder", "supervisor")
workflow.add_edge("Tester", "supervisor")
workflow.add_edge("Saver", "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in squad_members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("supervisor")

graph = workflow.compile()

In [186]:
from langchain_core.messages import HumanMessage

feature_request = """
Create a chain that does the following:
- Accept a string named answer as input
- Format a System and Human message using templates. The System message has output instructions via Pydantic. The Human message uses answer as context. Output instructions should require format to a Pydantic schema for hmw_question with a question (up to 10 words) and a role (either CMO, CTO, or CEO) 
- Pass the messages to OpenAI
- Parse the response using Pydantic
"""

inputs = {"messages": [HumanMessage(content=feature_request)]}
for output in graph.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'supervisor':
---
{'next': 'ContextRetriever'}

---

Retriever response:
[OpenAIToolAgentAction(tool='ContextRetriever', tool_input={'query': 'Create a chain that accepts a string named answer as input, formats a System and Human message using templates, and passes the messages to OpenAI.'}, log="\nInvoking: `ContextRetriever` with `{'query': 'Create a chain that accepts a string named answer as input, formats a System and Human message using templates, and passes the messages to OpenAI.'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nJNtzrsmgOeCWBURrcFfUOm8', 'function': {'arguments': '{"query": "Create a chain that accepts a string named answer as input, formats a System and Human message using templates, and passes the messages to OpenAI."}', 'name': 'ContextRetriever'}, 'type': 'function'}, {'id': 'call_DRXt1Mlb0hTQuJ2OMIU8JUJ5', 'function': {'arguments': '{"query": "Format a Pydantic schema for hmw_question with a ques

TypeError: list indices must be integers or slices, not str